In [ ]:
!pip install fi
!pip install timm

In [ ]:
!pip install wandb

In [ ]:
!pip install pytorch-gradcam
!pip install grad-cam

In [ ]:
!pip install unzip

In [ ]:
import timm
import fastai
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
import wandb
import fastai
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import PIL


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!unzip -qq '/content/drive/My Drive/Geolocation/50States10K.zip' -d '/content/geolocation'

In [ ]:

config = SimpleNamespace(
    batch_size=32,
    img_size=224,
    seed=42,
    pretrained=True,
    normalize=True,
    model_name="vit_small_patch32_224",
    epochs=20,  # FIX THIS LATER
    learning_rate=2e-3,
    resize="crop",
)



In [ ]:
def albumentations_transforms(x: PIL.Image.Image, img_size=224):
    transforms = A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.ImageCompression(quality_lower=99, quality_upper=100),
            A.ShiftScaleRotate(
                shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7
            ),
            A.Resize(img_size, img_size),
            A.Cutout(
                max_h_size=int(img_size * 0.4),
                max_w_size=int(img_size * 0.4),
                num_holes=1,
                p=0.5,
            ),
        ]
    )

    return PIL.Image.fromarray(transforms(image=np.array(x))["image"])

def load_test_data(augment):
    resize_method = (
        ResizeMethod.Crop
    )

    if augment:
        dls = fastai.vision.data.ImageDataLoaders.from_folder(
            "/content/geolocation/",
            valid_pct=0.99,
            seed=config.seed,
            bs=config.batch_size,
            item_tfms=[
                Resize(config.img_size, method=resize_method),
                albumentations_transforms,
            ],
        )
    else:
        dls = fastai.vision.data.ImageDataLoaders.from_folder(
            "/content/geolocation/",
            valid_pct=0.99,
            seed=config.seed,
            bs=config.batch_size,
            item_tfms=[Resize(config.img_size, method=resize_method)],
        )

    mean, std = (0.48145466, 0.4578275, 0.40821073), (
        0.26862954,
        0.26130258,
        0.27577711,
    )

    if config.normalize:
        dls.add_tfms([Normalize.from_stats(mean, std)], "after_batch")

    return dls

In [ ]:
import cv2
from pytorch_grad_cam import GradCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad, \
    DeepFeatureFactorization

from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    preprocess_image
from pytorch_grad_cam.ablation_layer import AblationLayerVit

In [ ]:
def create_learner(model_name, dls=None, augment=False, test_set=False):
    if dls is None:
        dls = load_test_data(augment)

    learn = vision_learner(
        dls,
        model_name,
        concat_pool=True,
    ).to_fp16()

    # To device
    learn.dls.device = device

    return learn
  
learn = create_learner("resnet18", augment=False, test_set=True)

In [ ]:
def load_wandb_model(model_num, learn):
    run = wandb.init()
    artifact = run.use_artifact(f"ben_z/geolocation/model:v{model_num}", type="model")
    artifact_dir = artifact.download()
    file = torch.load(os.path.join(artifact_dir, "model.pth"), map_location=device)

    # create a learner with vit_small_patch32_224
    learn.model.load_state_dict(file)

    # Move model to GPU
    learn.model = learn.model.to(device)
    learn.dls.device = device

    return learn

In [ ]:
wandb.finish()
learn = load_wandb_model(15, learn)


In [ ]:
model = learn.model[0].model
model.eval

In [ ]:
import matplotlib.pyplot as plt
from math import ceil, sqrt

def display_grid_image(class_image_list):
    # Calculate the grid size
    num_images = len(class_image_list)
    grid_size = ceil(sqrt(num_images))

    # Set up the matplotlib figure
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(10, 10))
    fig.subplots_adjust(hspace=0.2, wspace=0.2)

    # Flatten axes array if only 1 row or column
    if grid_size == 1:
        axes = [axes]

    # Iterate through the class-image list and add them to the grid
    for idx, (cls, img) in enumerate(class_image_list):
        row, col = divmod(idx, grid_size)
        ax = axes[row][col]
        ax.imshow(img)
        ax.set_title(cls)
        ax.axis("off")

    # Remove unused subplots
    for idx in range(num_images, grid_size * grid_size):
        row, col = divmod(idx, grid_size)
        fig.delaxes(axes[row][col])

    # Show the grid
    plt.show()


In [ ]:
def visualize(path, state=None):
  rgb_img = cv2.imread(path, 1)[:, :, ::-1]
  rgb_img = cv2.resize(rgb_img, (224, 224))
  rgb_img = np.float32(rgb_img) / 255
  input_tensor = preprocess_image(rgb_img, mean=[0.485, 0.456, 0.406], 
                                              std=[0.229, 0.224, 0.225])
  input_tensor = input_tensor.to(device)
  target_layers = [model.layer4[-1]]
  cam = EigenCAM(model=model, target_layers=target_layers, use_cuda=True)
  targets = None if not state else [ClassifierOutputTarget(list(learn.dls.vocab).index(state))]
  grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

  # In this example grayscale_cam has only one image in the batch:
  grayscale_cam = grayscale_cam[0, :]
  visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

  prediction = learn.predict(path)
  # print(prediction[0])
  # prediction = prediction.argmax(dim=-1)
  # Get the model's prediction
  # print(learn.dls.vocab[prediction])
  # print(prediction)
  return prediction[0], Image.fromarray(visualization)

In [ ]:
a,b = visualize('/content/e4.jpg')

b

In [ ]:
results = []
state = "Arizona"
for image in os.listdir(f"/content/geolocation/{state}")[:20]:
  results.append(visualize(f"/content/geolocation/{state}/{image}", state))

NameError: ignored

In [ ]:
display_grid_image(results)